In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
import numpy as np
import json
from tqdm.auto import tqdm
import random


import os
import sys
sys.path.append('..')

from relations import estimate
from util import model_utils
from dsets.counterfact import CounterFactDataset
from util import nethook
from operator import itemgetter

In [4]:
MODEL_NAME = "EleutherAI/gpt-j-6B"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B
mt = model_utils.ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=True, torch_dtype=torch.float16)

model = mt.model
tokenizer = mt.tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [5]:
jb_save_path = "gpt-j/jacobians_averaged"
os.makedirs(jb_save_path, exist_ok = True)

In [6]:
cut_off = 50 # minimum number of correct predictions

###########################################################################
relation_dct = {
    'P17'   : {'relation': '{} is located in the country of', 'correct_predict': None, 'cached_JB': None},
    'P641'  : {'relation': '{} plays the sport of', 'correct_predict': None, 'cached_JB': None},
    'P103'  : {'relation': 'The mother tongue of {} is', 'correct_predict': None, 'cached_JB': None},
    'P176'  : {'relation': '{} is produced by', 'correct_predict': None, 'cached_JB': None},
    'P140'  : {'relation': 'The official religion of {} is', 'correct_predict': None, 'cached_JB': None},
    'P1303' : {'relation': '{} plays the instrument', 'correct_predict': None, 'cached_JB': None},
    'P190'  : {'relation': 'What is the twin city of {}? It is', 'correct_predict': None, 'cached_JB': None},
    'P740'  : {'relation': '{} was founded in', 'correct_predict': None, 'cached_JB': None},
    'P178'  : {'relation': '{} was developed by', 'correct_predict': None, 'cached_JB': None},
    'P495'  : {'relation': '{}, that originated in the country of', 'correct_predict': None, 'cached_JB': None},
    'P127'  : {'relation': '{} is owned by', 'correct_predict': None, 'cached_JB': None},
    'P413'  : {'relation': '{} plays in the position of', 'correct_predict': None, 'cached_JB': None},
    'P39'   : {'relation': '{}, who holds the position of', 'correct_predict': None, 'cached_JB': None},
    'P159'  : {'relation': 'The headquarter of {} is located in', 'correct_predict': None, 'cached_JB': None},
    'P20'   : {'relation': '{} died in the city of', 'correct_predict': None, 'cached_JB': None},
    'P136'  : {'relation': 'What does {} play? They play', 'correct_predict': None, 'cached_JB': None},
    'P106'  : {'relation': 'The profession of {} is', 'correct_predict': None, 'cached_JB': None},
    'P30'   : {'relation': '{} is located in the continent of', 'correct_predict': None, 'cached_JB': None},
    'P937'  : {'relation': '{} worked in the city of', 'correct_predict': None, 'cached_JB': None},
    'P449'  : {'relation': '{} was released on', 'correct_predict': None, 'cached_JB': None},
    'P27'   : {'relation': '{} is a citizen of', 'correct_predict': None, 'cached_JB': None},
    'P101'  : {'relation': '{} works in the field of', 'correct_predict': None, 'cached_JB': None},
    'P19'   : {'relation': '{} was born in', 'correct_predict': None, 'cached_JB': None},
    'P37'   : {'relation': 'In {}, an official language is', 'correct_predict': None, 'cached_JB': None},
    'P138'  : {'relation': '{}, named after', 'correct_predict': None, 'cached_JB': None},
    'P131'  : {'relation': '{} is located in', 'correct_predict': None, 'cached_JB': None},
    'P407'  : {'relation': '{} was written in', 'correct_predict': None, 'cached_JB': None},
    'P108'  : {'relation': '{}, who is employed by', 'correct_predict': None, 'cached_JB': None},
    'P36'   : {'relation': 'The capital of {} is', 'correct_predict': None, 'cached_JB': None},
}
###########################################################################


root_path = "gpt-j"

pop_track = []
for relation in relation_dct:
    path = f"{root_path}/{relation}"
    with open(f"{path}/correct_prediction_{relation}.json") as f:
        correct_predictions = json.load(f)
    if(len(correct_predictions) < cut_off):
    # if "performance" not in os.listdir(path):
        print("skipped ", relation)
        pop_track.append(relation)
    
for r in pop_track:
    relation_dct.pop(r)

skipped  P1303
skipped  P190
skipped  P740
skipped  P413
skipped  P39
skipped  P136
skipped  P449
skipped  P138
skipped  P131
skipped  P407
skipped  P108


In [7]:
relation_id = "P106"

print(relation_dct[relation_id]['relation'])

The profession of {} is


In [8]:
def get_top_performers(relation_id, consider_top = 5):
    with open(f"gpt-j/{relation_id}/performance") as f:
        performance = json.load(f)
    performance.sort(key = itemgetter('p@3'), reverse=True)

    subject__top_performers = []
    object__top_performers = []
    top_performers = []

    for candidate in performance:
        subject = candidate['subject']
        try:
            sub_idx = candidate['misc']['h_info']['sub_index']
        except:
            sub_idx = candidate['misc']['h_info']['h_index']
        object = candidate['object']
        if(subject in subject__top_performers or object in object__top_performers):
            continue
        if(len(tokenizer(subject).input_ids) > 3):
            continue
        
        subject__top_performers.append(subject)
        object__top_performers.append(object)
        top_performers.append((
            subject, sub_idx, object #, candidate['p@3']
        ))

        if(len(top_performers) == consider_top):
            break
    return top_performers

top_performers = get_top_performers(relation_id, consider_top=10)
top_performers

[('Lucha Villa', 0, 'actor'), ('Cigoli', 0, 'architect')]

In [9]:
# best_performing_cases = {}

# for relation_id in relation_dct:
#     top_performers = get_top_performers(relation_id, consider_top=10)
#     cases_formatted = []
#     for subject, sub_idx, obj, perf in top_performers:
#         cases_formatted.append({
#             'subject': subject,
#             'sub_idx': sub_idx,
#             'object': obj,
#             'p@3_outof_50': perf
#         })
#     best_performing_cases[relation_id] = cases_formatted

# with open("best_performing_cases__relationwise__gpt-j.json", "w") as f:
#     json.dump(best_performing_cases, f)

In [10]:
r = relation_dct[relation_id]['relation']
r

'The profession of {} is'

In [11]:
def get_averaged_JB(top_performers, relation_prompt, num_icl = 3, calculate_at_lnf = False):
    try:
        jbs = []
        for s, s_idx, o in tqdm(top_performers):
            others = set(top_performers) - {(s, s_idx, o)}
            others = random.sample(list(others), k = min(num_icl, len(list(others)))) 
            prompt = ""
            prompt += "\n".join(relation_prompt.format(s_other) + f" {o_other}." for s_other, idx_other, o_other in others) + "\n"
            prompt += relation_prompt
            print("subject: ", s)
            print(prompt)

            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            jb = estimate.estimate_relation_operator(
                model, tokenizer,
                s, prompt,
                subject_token_index= s_idx,
                layer = 15,
                device = model.device,
                calculate_at_lnf = calculate_at_lnf
            )
            print(jb.weight.norm(), jb.bias.norm())
            print()
            jbs.append(jb)
        
        weight = torch.stack([jb.weight for jb in jbs]).mean(dim=0)
        bias  = torch.stack([jb.bias for jb in jbs]).mean(dim=0)

        return weight, bias
    except RuntimeError as e:
        if(str(e).startswith("CUDA out of memory")):
            print("CUDA out of memory")
        if(num_icl > 1):
            num_icl -= 1
            print("trying with smaller icl >> ", num_icl)
            return get_averaged_JB(top_performers, relation_prompt, num_icl, calculate_at_lnf)
        else:
            raise Exception("RuntimeError >> can't calculate Jacobian with minimum number of icl examples")

def get_multiple_averaged_JB(top_performers, relation_prompt, N = 3, num_icl = 3, calculate_at_lnf = False):
    weights_and_biases = []
    sample_size = min(len(top_performers), num_icl + 2)
    for _ in tqdm(range(N)):
        cur_sample = random.sample(top_performers, k = sample_size)
        weight, bias = get_averaged_JB(cur_sample, relation_prompt, num_icl, calculate_at_lnf)
        weights_and_biases.append({
            'weight': weight,
            'bias'  : bias
        })
    return weights_and_biases

In [12]:
# weights_and_biases = get_multiple_averaged_JB(top_performers, relation_prompt=r)

In [13]:
# len(weights_and_biases)

In [14]:

# weight, bias = get_averaged_JB(top_performers, r)
# relation = estimate.RelationOperator(
#     weight=weight,
#     bias=bias,
#     model=model,
#     tokenizer=tokenizer,
#     layer= 15 ,
#     relation= relation_dct[relation_id]['relation'],
# )

In [15]:
# test_subjects = [
#     "Hugh Jackman",
#     "Michael Phelps",
#     "Agatha Christie",
#     "Barack Obama",
#     "Sherlock Holmes",
#     "Alan Turing",
#     "Bill Gates",
#     "Michelangelo"
# ]

# for sub in test_subjects:
#     print(f"{sub} >> ", relation(sub, device= model.device))

In [16]:
# test_cases = [
#     # ("Statue of Liberty", -1, "United States"),
#     ("The Great Wall", -1, "China"),
#     ("Niagara Falls", -2, "Canada"),
#     ("Valdemarsvik", -1, "Sweden"),
#     ("Kyoto University", -2, "Japan"),
#     ("Hattfjelldal", -1, "Norway"),
#     ("Ginza", -1, "Japan"),
#     ("Sydney Hospital", -2, "Australia"),
#     ("Mahalangur Himal", -1, "Nepal"),
#     ("Higashikagawa", -1, "Japan"),
#     ("Trento", -1, "Italy"),
#     ("Taj Mahal", -1, "India"),
#     ("Hagia Sophia", -1, "Turkey"),
#     ("Colosseum", -1, "Italy"),
#     ("Mount Everest", -1, "Nepal"),
#     ("Valencia", -1, "Spain"),
#     ("Lake Baikal", -1, "Russia"),
#     ("Merlion Park", -1, "Singapore"),
#     ("Cologne Cathedral", -1, "Germany"),
#     ("Buda Castle", -1, "Hungary")
# ]

# def check_with_test_cases(relation_operator):
#     for subject, subject_token_index, target in test_cases:
#         objects = relation_operator(
#             subject,
#             subject_token_index=subject_token_index,
#             device=model.device,
#             return_top_k=5,
#         )
#         print(f"{subject}, target: {target}   ==>   predicted: {objects}")

# check_with_test_cases(relation)

In [17]:
# print(f"{jb_save_path}/{relation_id}.npz")

# np.savez(
#     f"{jb_save_path}/{relation_id}.npz", 
#     weight_and_bias = [{
#         'weight': weight.cpu().numpy(),
#         'bias'  : bias.cpu().numpy()
#     }],
#     allow_pickle = True
# )

In [18]:
# jb = np.load(f"{jb_save_path}/{relation_id}.npz", allow_pickle= True)
# list(jb.keys())

In [19]:
# jb['weight_and_bias'][0]['weight'].shape

In [20]:
len(relation_dct.keys())

18

In [21]:
jb_save_path = "gpt-j/jacobians_averaged_collection/before__ln_f"
os.makedirs(jb_save_path, exist_ok = True)

for relation_id in relation_dct:
    print("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz")
    print(relation_id, relation_dct[relation_id]['relation'])
    print("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz")
    path = f"{root_path}/{relation_id}"
    try:
        with open(f"{path}/correct_prediction_{relation_id}.json") as f:
            correct_predictions = json.load(f)
            print(len(correct_predictions))
    except:
        print(f"Error opening correct prediction {relation_id} (maybe the scan skipped this relation?)")
        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        continue
        
    if(len(correct_predictions) < cut_off):
        print(f"skipped {relation_id} >> ", len(correct_predictions))
        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        continue

    top_performers = get_top_performers(relation_id, consider_top=10)

    relation_prompt = relation_dct[relation_id]['relation']
    print(relation_prompt)

    # weight, bias = get_averaged_JB(top_performers, relation_prompt)
    weights_and_biases = get_multiple_averaged_JB(
        top_performers, 
        relation_prompt=relation_prompt, 
        N = 10, 
        calculate_at_lnf=False
    )

    save_path = f"{jb_save_path}/{relation_id}.npz"
    print("Saving weights and biases >> ", save_path)
    np.savez(
        save_path, 
        # JB = [{
        #     'weight': weight.cpu().numpy(),
        #     'bias'  : bias.cpu().numpy()
        # }],
        weights_and_biases = weights_and_biases,
        allow_pickle = True
    )
    print("----------------------------------------------------")


zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
P17 {} is located in the country of
zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
404
{} is located in the country of


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

subject:  Haut Atlas
Marugame is located in the country of Japan.
Sabratha is located in the country of Libya.
Southern Savonia is located in the country of Finland.
{} is located in the country of
tensor(12.1172, device='cuda:0', dtype=torch.float16) tensor(269.7500, device='cuda:0', dtype=torch.float16)

subject:  Southern Savonia
Umarex is located in the country of Germany.
Sabratha is located in the country of Libya.
Haut Atlas is located in the country of Morocco.
{} is located in the country of
tensor(14.3047, device='cuda:0', dtype=torch.float16) tensor(238.3750, device='cuda:0', dtype=torch.float16)

subject:  Marugame
Haut Atlas is located in the country of Morocco.
Sabratha is located in the country of Libya.
Umarex is located in the country of Germany.
{} is located in the country of
tensor(41.1875, device='cuda:0', dtype=torch.float16) tensor(238.8750, device='cuda:0', dtype=torch.float16)

subject:  Sabratha
Southern Savonia is located in the country of Finland.
Marugame i

  0%|          | 0/5 [00:00<?, ?it/s]

...

subject:  Lebanon
The capital of Campania is Naples.
The capital of Saxony is Dresden.
The capital of Norway is Oslo.
The capital of {} is
tensor(30.3750, device='cuda:0', dtype=torch.float16) tensor(190.7500, device='cuda:0', dtype=torch.float16)

subject:  Campania
The capital of Saxony is Dresden.
The capital of Norway is Oslo.
The capital of Lebanon is Beirut.
The capital of {} is
tensor(31.5469, device='cuda:0', dtype=torch.float16) tensor(183.7500, device='cuda:0', dtype=torch.float16)

subject:  Lazio
The capital of Norway is Oslo.
The capital of Lebanon is Beirut.
The capital of Saxony is Dresden.
The capital of {} is
tensor(26.2344, device='cuda:0', dtype=torch.float16) tensor(203.2500, device='cuda:0', dtype=torch.float16)

subject:  Norway
The capital of Lebanon is Beirut.
The capital of Saxony is Dresden.
The capital of Lazio is Rome.
The capital of {} is
tensor(22.1250, device='cuda:0', dtype=torch.float16) tensor(213.2500, device='cuda:0', dtype=torch.float16)

subject:  

  0%|          | 0/5 [00:00<?, ?it/s]

subject:  Lebanon
The capital of Peru is Lima.
The capital of Portugal is Lisbon.
The capital of United Arab Republic is Cairo.
The capital of {} is
tensor(29.5469, device='cuda:0', dtype=torch.float16) tensor(205.1250, device='cuda:0', dtype=torch.float16)

subject:  Peru
The capital of United Arab Republic is Cairo.
The capital of Portugal is Lisbon.
The capital of South Yemen is Aden.
The capital of {} is
tensor(28.3750, device='cuda:0', dtype=torch.float16) tensor(200.7500, device='cuda:0', dtype=torch.float16)

subject:  South Yemen
The capital of United Arab Republic is Cairo.
The capital of Peru is Lima.
The capital of Portugal is Lisbon.
The capital of {} is
tensor(41.5000, device='cuda:0', dtype=torch.float16) tensor(199.2500, device='cuda:0', dtype=torch.float16)

subject:  Portugal
The capital of United Arab Republic is Cairo.
The capital of Lebanon is Beirut.
The capital of South Yemen is Aden.
The capital of {} is
tensor(22.5469, device='cuda:0', dtype=torch.float16) tenso

  0%|          | 0/5 [00:00<?, ?it/s]

subject:  Lazio
The capital of Saxony is Dresden.
The capital of Peru is Lima.
The capital of South Yemen is Aden.
The capital of {} is
tensor(30.0312, device='cuda:0', dtype=torch.float16) tensor(191.7500, device='cuda:0', dtype=torch.float16)

subject:  Norway
The capital of South Yemen is Aden.
The capital of Peru is Lima.
The capital of Saxony is Dresden.
The capital of {} is
tensor(21.9531, device='cuda:0', dtype=torch.float16) tensor(215.6250, device='cuda:0', dtype=torch.float16)

subject:  South Yemen
The capital of Peru is Lima.
The capital of Lazio is Rome.
The capital of Saxony is Dresden.
The capital of {} is
tensor(38.6562, device='cuda:0', dtype=torch.float16) tensor(199.2500, device='cuda:0', dtype=torch.float16)

subject:  Peru
The capital of Norway is Oslo.
The capital of South Yemen is Aden.
The capital of Saxony is Dresden.
The capital of {} is
tensor(29.8750, device='cuda:0', dtype=torch.float16) tensor(199.6250, device='cuda:0', dtype=torch.float16)

subject:  Saxo